In [ ]:
!pip install git+https://github.com/openai/whisper.git
!pip install transformers datasets
!sudo apt update && sudo apt install ffmpeg -y


  Cloning https://github.com/openai/whisper.git to /tmp/pip-req-build-qybx3763
  Running command git clone --filter=blob:none --quiet https://github.com/openai/whisper.git /tmp/pip-req-build-qybx3763
  Resolved https://github.com/openai/whisper.git to commit c0d2f624c09dc18e709e37c2ad90c039a4eb72a2
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 981.2 kB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 96.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 80.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 59.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 1.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 5.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 13.7 MB/s eta 0:00:00
   ━━━

In [ ]:
from google.colab import files
uploaded = files.upload()  # Upload a .mp3, .m4a, .wav, etc.

import os
for fname in uploaded.keys():
    audio_file = fname
    print(f"Uploaded: {audio_file}")


Saving Special Meeting Audio File - April 29, 2025.mp3 to Special Meeting Audio File - April 29, 2025.mp3
Uploaded: Special Meeting Audio File - April 29, 2025.mp3


In [ ]:
import whisper

model = whisper.load_model("base")  # Use 'base' for speed, 'small'/'medium' for quality
result = model.transcribe(audio_file)
transcript = result['text']
print("Transcription:\n", transcript)


100%|████████████████████████████████████████| 139M/139M [00:01<00:00, 119MiB/s]
/usr/local/lib/python3.11/dist-packages/whisper/transcribe.py:132: UserWarning: FP16 is not supported on CPU; using FP32 instead
  warnings.warn("FP16 is not supported on CPU; using FP32 instead")


Transcription:
  I'll start again. This is the meeting for the Armour Springfield special meeting to discuss our 2025 financial plan. Today's date is April 29th at 2025 and the time is exactly 6 p.m. I'm Mayor Patrick Terriam for the Armour Springfield. All council is present to my right. The sending order is a Glen Fuel, Andy Kaczynski, Mark Miller and Melinda Warren. If I can get a adoption of the agenda there, move on to seconder please. Melinda and Glen, in addition to council at all. I see none. If I can get a mover or those in support of the adoption of the agenda. Andy, that would be unanimous and is there for or past. We'll get into 4.1 that's adoption of the 2025 financial plan. Can they get a mover and a seconder for that as well please. Melinda and Patrick. I will also introduce the council of the Armour Springfield. I will also introduce the council of the Armour Springfield, the 2025 financial plan consisting of one and operating budget to a capital budget. Three, an estim

In [ ]:
from transformers import pipeline

# Load summarizer model
summarizer = pipeline("summarization", model="facebook/bart-large-cnn")

# Helper: Split text into chunks (approx 700 words)
def split_text(text, max_words=700):
    words = text.split()
    return [' '.join(words[i:i+max_words]) for i in range(0, len(words), max_words)]

# Split transcript if needed
chunks = split_text(transcript)

# Summarize each chunk
summaries = []
for i, chunk in enumerate(chunks):
    print(f"\n🔹 Summarizing chunk {i+1}/{len(chunks)}...")
    summary = summarizer(chunk, max_length=130, min_length=30, do_sample=False)[0]['summary_text']
    summaries.append(summary)

# Combine all summaries into one
final_summary = "\n".join(summaries)
print("\n✅ Final Summary:\n", final_summary)


Device set to use cpu



🔹 Summarizing chunk 1/2...

🔹 Summarizing chunk 2/2...

✅ Final Summary:
 Mayor Patrick Terriam: This is the meeting for the Armour Springfield special meeting to discuss our 2025 financial plan. Council must buy bylaws, set a rate of taxation and adopt its operating budget for the year.
The Council of the Armistrowing Field adopt the 2025 financial plan. Councilor Melinda and Melinda Warren and Deputy Mayor Fuel and at 617 PM, the meeting is adjourned.


In [ ]:
import json
from datetime import datetime

memory = {
    "timestamp": str(datetime.now()),
    "audio_file": audio_file,
    "transcript": transcript,
    "summary": final_summary
}

# Save to file
with open("memory.json", "w") as f:
    json.dump(memory, f, indent=4)

print("✅ Saved to memory.json")


✅ Saved to memory.json


In [ ]:
import os
import json
from datetime import datetime

# Create a memory folder locally (won't persist across sessions)
os.makedirs("memory_local", exist_ok=True)

# Save to a unique filename with timestamp
timestamp = datetime.now().strftime('%Y-%m-%d_%H-%M-%S')
filename = f"memory_local/memory_{timestamp}.json"

with open(filename, "w") as f:
    json.dump(memory, f, indent=4)

print(f"✅ Saved memory to {filename}")


✅ Saved memory to memory_local/memory_2025-07-04_10-17-39.json


In [ ]:
import os
import json
from datetime import datetime

# Prepare current memory data
memory = {
    "timestamp": str(datetime.now()),
    "transcript": transcript,
    "summary": final_summary
}

# Load memory_log.json if it exists; else start fresh
if os.path.exists("memory_log.json"):
    with open("memory_log.json", "r") as f:
        memory_log = json.load(f)
else:
    memory_log = []

# Append this new memory
memory_log.append(memory)

# Save back to memory_log.json
with open("memory_log.json", "w") as f:
    json.dump(memory_log, f, indent=4)

print("✅ Memory saved and logged to memory_log.json")


✅ Memory saved and logged to memory_log.json


In [ ]:
# Load memory log again
with open("memory_log.json", "r") as f:
    memory_log = json.load(f)

# Define search function
def search_memories(keyword, search_in="summary"):
    print(f"\n🔎 Searching for '{keyword}' in {search_in}...\n")
    results = []
    for mem in memory_log:
        text = mem.get(search_in, "").lower()
        if keyword.lower() in text:
            results.append(mem)
    return results

# Input and search
keyword = input("Enter keyword to search: ")
results = search_memories(keyword)

# Show results
print(f"\n✅ Found {len(results)} result(s):\n")
for i, mem in enumerate(results):
    print(f"🔹 {i+1}. Timestamp: {mem['timestamp']}")
    print(f"📝 Summary: {mem['summary'][:300]}...\n")


Enter keyword to search: mayor

🔎 Searching for 'mayor' in summary...


✅ Found 1 result(s):

🔹 1. Timestamp: 2025-07-04 10:18:38.071901
📝 Summary: Mayor Patrick Terriam: This is the meeting for the Armour Springfield special meeting to discuss our 2025 financial plan. Council must buy bylaws, set a rate of taxation and adopt its operating budget for the year.
The Council of the Armistrowing Field adopt the 2025 financial plan. Councilor Melind...



In [ ]:
# Optional: Add simple tags based on keywords
def auto_tag(text):
    tags = []
    if "meeting" in text: tags.append("meeting")
    if "project" in text: tags.append("project")
    if "lecture" in text: tags.append("lecture")
    if "reminder" in text: tags.append("reminder")
    return tags

memory["tags"] = auto_tag(transcript)


In [6]:
!pip install -q pydub
!apt install ffmpeg -y


Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
ffmpeg is already the newest version (7:4.4.2-0ubuntu0.22.04.1).
0 upgraded, 0 newly installed, 0 to remove and 35 not upgraded.


In [8]:
# 🔧 Step 1: Install required packages
!pip install -q gradio transformers openai-whisper pydub
!apt install ffmpeg -y


Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
ffmpeg is already the newest version (7:4.4.2-0ubuntu0.22.04.1).
0 upgraded, 0 newly installed, 0 to remove and 35 not upgraded.


In [9]:
# 🧠 Step 2: Imports and setup
import gradio as gr
import whisper
from transformers import pipeline
import os
import json
from datetime import datetime
from pydub import AudioSegment
import tempfile

# Load models once
whisper_model = whisper.load_model("base")
summarizer = pipeline("summarization", model="facebook/bart-large-cnn")


Device set to use cpu


In [10]:
# 🧩 Step 3: Process uploaded audio (.mp3, .wav, etc.)
def process_audio(audio):
    try:
        # ✅ Convert to WAV using pydub (handles .mp3 and others)
        temp_wav = tempfile.NamedTemporaryFile(suffix=".wav", delete=False)
        audio_data = AudioSegment.from_file(audio)
        audio_data.export(temp_wav.name, format="wav")

        # 🧠 Transcribe with Whisper
        result = whisper_model.transcribe(temp_wav.name)
        transcript = result["text"]

        # ❗ If no speech detected
        if not transcript.strip():
            return "❌ No speech detected or audio is silent."

        # 📝 Summarize
        summary = summarizer(transcript, max_length=130, min_length=30, do_sample=False)[0]["summary_text"]

        # 💾 Save memory
        memory = {
            "timestamp": str(datetime.now()),
            "transcript": transcript,
            "summary": summary
        }

        # Append to memory log
        if os.path.exists("memory_log.json"):
            with open("memory_log.json", "r") as f:
                memory_log = json.load(f)
        else:
            memory_log = []

        memory_log.append(memory)

        with open("memory_log.json", "w") as f:
            json.dump(memory_log, f, indent=4)

        return f"📜 Transcript (preview):\n{transcript[:400]}...\n\n📝 Summary:\n{summary}"

    except Exception as e:
        return f"❌ Error: {str(e)}"


In [4]:
!pip install -U openai-whisper transformers pydub gradio
!apt install ffmpeg  # Required for audio conversion



     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 803.2/803.2 kB 9.9 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 40.9/40.9 kB 2.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.8/10.8 MB 67.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 54.3/54.3 MB 15.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 323.9/323.9 kB 19.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 61.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 33.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 39.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 767.0 kB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 6.9 MB

Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
ffmpeg is already the newest version (7:4.4.2-0ubuntu0.22.04.1).
^C


In [2]:
# Install (only needed once per runtime)
# !pip install -U openai-whisper transformers pydub gradio
# !apt install ffmpeg

import gradio as gr
import whisper
from transformers import pipeline
from pydub import AudioSegment
import os
import tempfile

# Load models once
whisper_model = whisper.load_model("base")  # use "small" or "medium" for better accuracy
summarizer = pipeline("summarization", model="facebook/bart-large-cnn")

def process_audio(audio_path):
    # Convert if needed to .wav
    audio = AudioSegment.from_file(audio_path)
    with tempfile.NamedTemporaryFile(suffix=".wav", delete=False) as tmp_wav:
        wav_path = tmp_wav.name
        audio.export(wav_path, format="wav")

    # Transcribe
    result = whisper_model.transcribe(wav_path)
    transcript = result["text"]

    # Summarize in chunks if long
    max_chunk = 1000
    transcript_chunks = [transcript[i:i+max_chunk] for i in range(0, len(transcript), max_chunk)]

    summary = ""
    for chunk in transcript_chunks:
        part = summarizer(chunk, max_length=130, min_length=30, do_sample=False)[0]['summary_text']
        summary += part + "\n"

    output = f"📝 **Transcript:**\n{transcript}\n\n🧠 **Summary:**\n{summary}"
    return output

# Gradio Interface
app = gr.Interface(
    fn=process_audio,
    inputs=gr.Audio(type="filepath", label="🎧 Upload Audio File"),
    outputs=gr.Textbox(label="📄 Transcript + Summary"),
    title="🧠 PIH Voice Summarizer",
    theme="dark"
)

# Launch
app.launch(share=True)


Device set to use cpu
/usr/local/lib/python3.11/dist-packages/gradio/blocks.py:1180: UserWarning: Cannot load dark. Caught Exception: 404 Client Error: Not Found for url: https://huggingface.co/api/spaces/dark (Request ID: Root=1-6867f689-0a1c15f16b34eaea41469bb6;53edccaa-39c5-4abe-b9e6-2269b7c747b5)

Sorry, we can't find the page you are looking for.
  warnings.warn(f"Cannot load {theme}. Caught Exception: {str(e)}")


Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://88ddf0d4a510a50478.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)


In [5]:
def process_audio(audio_path):
    import os
    from pydub import AudioSegment

    # Convert to .wav if needed
    if not audio_path.endswith(".wav"):
        sound = AudioSegment.from_file(audio_path)
        audio_path = audio_path.replace(os.path.splitext(audio_path)[1], ".wav")
        sound.export(audio_path, format="wav")

    # Transcribe with Whisper
    result = whisper_model.transcribe(audio_path)
    transcript = result["text"]

    # Summarize
    summary = summarizer(transcript, max_length=100, min_length=30, do_sample=False)[0]["summary_text"]

    # Save both to text files
    transcript_file = "transcript.txt"
    summary_file = "summary.txt"

    with open(transcript_file, "w") as f:
        f.write(transcript)
    with open(summary_file, "w") as f:
        f.write(summary)

    # Return display and downloadable files
    display_text = f"📄 **Transcript:**\n{transcript}\n\n💡 **Summary:**\n{summary}"
    return display_text, transcript_file, summary_file


In [6]:
gr.Interface(
    fn=process_audio,
    inputs=gr.Audio(type="filepath", label="🎧 Upload Audio File"),
    outputs=[
        gr.Textbox(label="📄 Transcript + Summary"),
        gr.File(label="⬇️ Download Transcript (.txt)"),
        gr.File(label="⬇️ Download Summary (.txt)")
    ],
    title="🎙️ Audio Transcriber + Summary",
    description="Upload audio to get a transcript and summary. Then download them as .txt files."
).launch()


It looks like you are running Gradio on a hosted Jupyter notebook, which requires `share=True`. Automatically setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://85eeee508fef157fdf.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)


In [7]:
import os
import json
from datetime import datetime

# Set up memory storage
memory_dir = "pih_memory"
os.makedirs(memory_dir, exist_ok=True)

log_file = os.path.join(memory_dir, "memory_log.json")

# Initialize memory log if it doesn't exist
if not os.path.exists(log_file):
    with open(log_file, "w") as f:
        json.dump([], f)


In [9]:
def process_audio(audio_path):
    import whisper
    from transformers import pipeline
    import os
    import json
    from datetime import datetime

    # Load Whisper model
    model = whisper.load_model("base")
    result = model.transcribe(audio_path)
    transcript = result["text"]

    # Summarize
    summarizer = pipeline("summarization", model="facebook/bart-large-cnn")
    summary = summarizer(transcript, max_length=130, min_length=30, do_sample=False)
    summary_text = summary[0]['summary_text']

    # Save transcript and summary to .txt files
    with open("transcript.txt", "w") as f:
        f.write(transcript)
    with open("summary.txt", "w") as f:
        f.write(summary_text)

    # --- MEMORY STORAGE ---
    memory_dir = "pih_memory"
    os.makedirs(memory_dir, exist_ok=True)
    log_file = os.path.join(memory_dir, "memory_log.json")

    # If memory log doesn't exist, initialize it
    if not os.path.exists(log_file):
        with open(log_file, "w") as f:
            json.dump([], f)

    # Create memory entry
    memory_entry = {
        "id": str(datetime.now().timestamp()),
        "date": str(datetime.now().date()),
        "tags": ["meeting", "note"],  # basic for now
        "transcript": transcript,
        "summary": summary_text
    }

    # Append to memory log
    with open(log_file, "r") as f:
        logs = json.load(f)
    logs.append(memory_entry)
    with open(log_file, "w") as f:
        json.dump(logs, f, indent=2)

    return transcript + "\n\n📄 Summary:\n" + summary_text, "transcript.txt", "summary.txt"


In [11]:
import gradio as gr
import json
import os

# 🔍 Memory Search Function
def search_memories(keyword, search_in="summary"):
    memory_dir = "pih_memory"
    log_file = os.path.join(memory_dir, "memory_log.json")

    if not os.path.exists(log_file):
        return "⚠️ No memory log found."

    with open(log_file, "r") as f:
        logs = json.load(f)

    keyword = keyword.lower()
    results = []

    for entry in logs:
        content = entry.get(search_in, "").lower()
        if keyword in content:
            result_text = f"🗓 Date: {entry['date']}\n🔹 Summary: {entry['summary']}\n"
            results.append(result_text)

    return "\n\n---\n\n".join(results) if results else "🔍 No matches found."


# 🚀 Launch Gradio Interface
with gr.Blocks() as app:
    gr.Markdown("## 🧠 Personal Information Hub (PIH)")

    with gr.Tab("🎤 Transcribe + Summarize"):
        audio_input = gr.Audio(type="filepath", label="Upload Audio")
        output_text = gr.Textbox(label="📄 Transcript + Summary", lines=10)
        transcript_file = gr.File(label="📥 Download Transcript")
        summary_file = gr.File(label="📥 Download Summary")
        button = gr.Button("Transcribe + Summarize")

        button.click(
            fn=process_audio,
            inputs=[audio_input],
            outputs=[output_text, transcript_file, summary_file]
        )

    with gr.Tab("🔎 Search Memory"):
        search_input = gr.Textbox(label="Enter keyword (e.g., meeting, lecture, task)")
        search_results = gr.Textbox(label="Matching Summaries", lines=10)
        search_button = gr.Button("Search Memory")

        search_button.click(
            fn=search_memories,
            inputs=[search_input],
            outputs=[search_results]
        )

app.launch()


It looks like you are running Gradio on a hosted Jupyter notebook, which requires `share=True`. Automatically setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://b025e896e385036734.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)
